In [1]:
import numpy as np
from collections import Counter

In [ ]:
class tree_node: 
    def __init__(self,condition , left_node  , right_node , feature , pred_data, leaf_node = False):
        self.feature = feature
        self.condition = condition
        self.right_node = right_node
        self.left_node = left_node

        self.pred_data = pred_data 
        self.is_leaf_node = leaf_node 
    

class decision_tree: 
    def __init__(self,max_depth , max_purity):
        self.max_depth = max_depth 

        self.max_purity = max_purity
        self.root_node = None

    def entropy(self,y_node): 
        count_object = Counter(y_node).items()
        total = len(y_node)
        entropy = 0 
        for item,frequency in count_object: 
            entropy -= (frequency/total) * np.log2(frequency/total)
        
        return entropy 

    def data_split (self , x , y , condition , feature): 
        x_left , y_left , x_right , y_right = [] , [] ,[] ,[]
        for index, datapoint in enumerate(x[:,feature]): 
            if(datapoint > condition): 
                x_right.append(x[index])
                y_right.append(y[index])
            else: 
                x_left.append(x[index])
                y_left.append(y[index])
        return (np.array(x_left), np.array(y_left),
                np.array(x_right), np.array(y_right))

    def build_tree(self , x , y , current_depth):
        no_of_datapoionts , no_of_features = x.shape

        if(current_depth >= self.max_depth or self.entropy(y) <= self.max_purity): 
            leaf_node = tree_node(None , None , None ,None , Counter(y).most_common(1)[0][0], True)
            return leaf_node

        best_feature_index , best_condition_value = None , None
        max_information_gain = 0 ; 
        for features_index in range(len(x[0])) :
            for value  in x[:,features_index]: 
                x_left_data , y_left_data , x_right_data , y_right_data = self.data_split(x,y,value,features_index)
                information_gain = self.entropy(y) - (len(y_left_data)/len(y)) * self.entropy(y_left_data) - (len(y_right_data)/len(y)) * self.entropy(y_right_data)

                if(information_gain > max_information_gain): 
                    max_information_gain = information_gain 
                    best_feature_index = features_index 
                    best_condition_value = value

        if best_feature_index is None or max_information_gain == 0:
            return tree_node(None, None, None, None, Counter(y).most_common(1)[0][0], True)
        
        x_left_data , y_left_data , x_right_data , y_right_data = self.data_split(x,y,best_condition_value , best_feature_index)
        left_node = self.build_tree(x_left_data , y_left_data , current_depth+1)
        right_node = self.build_tree(x_right_data , y_right_data , current_depth+1)

        current_node = tree_node(best_condition_value , left_node , right_node , best_feature_index , None)
        return current_node 
    def train(self , x_data , y_data): 
        self.root_node = self.build_tree(x_data,y_data,0)
        
    def predict(self, x_data):
        return [self._predict_single(x) for x in x_data]

    def _predict_single(self, x):
        node = self.root_node
        while not node.is_leaf_node:
            if x[node.feature] > node.condition:
                node = node.right_node
            else:
                node = node.left_node
        return node.pred_data
        
            





         

